## regression_normalization

-   回帰
-   正規化あり
-   シャッフルでの効果検証
-   sklearn の MinMaxScaler による正規化


### 準備


In [22]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler

In [23]:
DATA_DIR = './dms_data'
N_TRIALS = 5

In [24]:

def load_and_preprocess_data(file_path):
  """CSVを読み込んで前処理する関数。"""
  df = pd.read_csv(file_path)
  df = df.drop(['timestamp'], axis=1)
  return df.dropna()


def get_data_from_directory(directory_path):
  """指定ディレクトリのCSV全部を読み込んで結合する関数。"""
  files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
  data_frames = [load_and_preprocess_data(os.path.join(directory_path, file)) for file in files]
  return pd.concat(data_frames, ignore_index=True)


# 特徴量を定義
features = [
    'm_speed', 'm_speed_var_480', 'm_speed_stddev_480', 'm_acceleration',
    'm_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk',
    'm_jerk_var_480', 'm_jerk_stddev_480', 'oss', 'Sleepiness'
]


def evaluate_model(model, X, y):
  """モデルの評価。RMSE"""
  predictions = model.predict(X)
  return np.sqrt(mean_squared_error(y, predictions))


def printResults(rmse_results, title):
  mean_rmse = np.mean(rmse_results)
  median_rmse = np.median(rmse_results)
  variance_rmse = np.var(rmse_results)
  std_rmse = np.std(rmse_results)
  min_rmse = np.min(rmse_results)
  max_rmse = np.max(rmse_results)
  print(f"[{title}]RMSEの平均値: {mean_rmse}")
  print(f"[{title}]RMSEの中央値: {median_rmse}")
  print(f"[{title}]RMSEの分散: {variance_rmse}")
  print(f"[{title}]RMSEの標準偏差: {std_rmse}")
  print(f"[{title}]RMSEの最小値: {min_rmse}")
  print(f"[{title}]RMSEの最大値: {max_rmse}")


def get_data_from_directory_with_scaling(directory_path, scaler=None):
  """指定ディレクトリのCSV全部を読み込んで結合し、スケーリングする関数。"""
  files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
  data_frames = [load_and_preprocess_data(os.path.join(directory_path, file)) for file in files]
  combined_df = pd.concat(data_frames, ignore_index=True)

  if scaler is not None:
    # 既存のスケーラーを使用してデータを変換
    combined_df[features] = scaler.transform(combined_df[features])
  else:
    # 新しいスケーラーをフィットしてデータを変換
    scaler = StandardScaler()
    combined_df[features] = scaler.fit_transform(combined_df[features])

  return combined_df, scaler

In [25]:
# トレインデータとテストデータを読み込む
# train = get_data_from_directory(os.path.join(DATA_DIR, 'train'))
# test = get_data_from_directory(os.path.join(DATA_DIR, 'test'))
train, scaler = get_data_from_directory_with_scaling(os.path.join(DATA_DIR, 'train'))
test, _ = get_data_from_directory_with_scaling(os.path.join(DATA_DIR, 'test'), scaler=scaler)

# oss_variance = train_data['oss'].var()
# sleepiness_variance = train_data['Sleepiness'].var()

# print(f"ossの分散: {oss_variance}")
# print(f"Sleepinessの分散: {sleepiness_variance}")
# print(train_data.head())
# print(train_data.describe())

### LightGBM


#### 正規化導入前


In [26]:
def train_lightgbm_full(X, y):
  """Train LightGBM model on the full training data."""
  train_data = lgb.Dataset(X, label=y)
  params = {
      'objective': 'regression',
      'metric': 'rmse',
      'verbose': -1
  }
  model = lgb.train(params, train_data)
  return model


def evaluate_on_test(model, X_test, y_test):
  """Evaluate model on the test data."""
  predictions = model.predict(X_test)
  return np.sqrt(mean_squared_error(y_test, predictions))


def evaluate_trials_with_test(X, y, X_test, y_test):
  rmses_train = []
  rmses_test = []
  for _ in range(N_TRIALS):
    model = train_lightgbm_full(X, y)
    rmse_train = evaluate_model(model, X, y)
    rmse_test = evaluate_on_test(model, X_test, y_test)
    rmses_train.append(rmse_train)
    rmses_test.append(rmse_test)
  return rmses_train, rmses_test

In [27]:
train_data = get_data_from_directory(os.path.join(DATA_DIR, 'train'))
X = train_data.drop(['oss', 'Sleepiness'], axis=1)

test_data = get_data_from_directory(os.path.join(DATA_DIR, 'test'))
X_test = test_data.drop(['oss', 'Sleepiness'], axis=1)

In [28]:
oss_rmses_train, oss_rmses_test = evaluate_trials_with_test(X, train_data['oss'], X_test, test_data['oss'])
printResults(oss_rmses_train, 'lgbm-oss-train')
print('-------------------------')
printResults(oss_rmses_test, 'lgbm-oss-test')

[lgbm-oss-train]RMSEの平均値: 0.12457814822031583
[lgbm-oss-train]RMSEの中央値: 0.12457814822031583
[lgbm-oss-train]RMSEの分散: 0.0
[lgbm-oss-train]RMSEの標準偏差: 0.0
[lgbm-oss-train]RMSEの最小値: 0.12457814822031583
[lgbm-oss-train]RMSEの最大値: 0.12457814822031583
-------------------------
[lgbm-oss-test]RMSEの平均値: 0.541056204141207
[lgbm-oss-test]RMSEの中央値: 0.541056204141207
[lgbm-oss-test]RMSEの分散: 0.0
[lgbm-oss-test]RMSEの標準偏差: 0.0
[lgbm-oss-test]RMSEの最小値: 0.541056204141207
[lgbm-oss-test]RMSEの最大値: 0.541056204141207


In [29]:
sleepiness_rmses_train, sleepiness_rmses_test = evaluate_trials_with_test(X, train_data['Sleepiness'], X_test, test_data['Sleepiness'])
printResults(sleepiness_rmses_train, 'lgbm-sleepiness-train')
print('-------------------------')
printResults(sleepiness_rmses_test, 'lgbm-sleepiness-test')

[lgbm-sleepiness-train]RMSEの平均値: 0.22518856302682227
[lgbm-sleepiness-train]RMSEの中央値: 0.22518856302682227
[lgbm-sleepiness-train]RMSEの分散: 0.0
[lgbm-sleepiness-train]RMSEの標準偏差: 0.0
[lgbm-sleepiness-train]RMSEの最小値: 0.22518856302682227
[lgbm-sleepiness-train]RMSEの最大値: 0.22518856302682227
-------------------------
[lgbm-sleepiness-test]RMSEの平均値: 1.3758729110252772
[lgbm-sleepiness-test]RMSEの中央値: 1.3758729110252772
[lgbm-sleepiness-test]RMSEの分散: 0.0
[lgbm-sleepiness-test]RMSEの標準偏差: 0.0
[lgbm-sleepiness-test]RMSEの最小値: 1.3758729110252772
[lgbm-sleepiness-test]RMSEの最大値: 1.3758729110252772


In [30]:
from sklearn.model_selection import KFold


def evaluate_with_kfold(X, y, n_splits=5):
  """5分割交差検証を使用してモデルを評価する関数"""
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
  rmses = []

  for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = train_lightgbm_full(X_train, y_train)
    rmse = evaluate_on_test(model, X_test, y_test)
    rmses.append(rmse)

  return rmses


all_data = pd.concat([train, test], ignore_index=True)

X_all = all_data.drop(["oss", "Sleepiness"], axis=1)
y_all_oss = all_data["oss"]
y_all_sleepiness = all_data["Sleepiness"]

oss_kfold_rmses = evaluate_with_kfold(X_all, y_all_oss)
printResults(oss_kfold_rmses, "lgbm-oss-kfold")

print('-------------------------')

sleepiness_kfold_rmses = evaluate_with_kfold(X_all, y_all_sleepiness)
printResults(sleepiness_kfold_rmses, "lgbm-sleepiness-kfold")

[lgbm-oss-kfold]RMSEの平均値: 0.419917153473233
[lgbm-oss-kfold]RMSEの中央値: 0.4135138551360697
[lgbm-oss-kfold]RMSEの分散: 0.0005697273320027021
[lgbm-oss-kfold]RMSEの標準偏差: 0.02386896168673246
[lgbm-oss-kfold]RMSEの最小値: 0.3987296327306805
[lgbm-oss-kfold]RMSEの最大値: 0.46619589059527167
-------------------------
[lgbm-sleepiness-kfold]RMSEの平均値: 0.3267308093924702
[lgbm-sleepiness-kfold]RMSEの中央値: 0.31569918359998
[lgbm-sleepiness-kfold]RMSEの分散: 0.0003490758585121301
[lgbm-sleepiness-kfold]RMSEの標準偏差: 0.018683571888483477
[lgbm-sleepiness-kfold]RMSEの最小値: 0.3077176879632781
[lgbm-sleepiness-kfold]RMSEの最大値: 0.3498232614340698


#### 正規化導入


In [31]:
from sklearn.preprocessing import MinMaxScaler

In [32]:

train_data = get_data_from_directory(os.path.join(DATA_DIR, 'train'))
test_data = get_data_from_directory(os.path.join(DATA_DIR, 'test'))


def normalize_data(series):
  scaler = MinMaxScaler()
  series = series.values.reshape(-1, 1)
  series = scaler.fit_transform(series)
  series = series.ravel()
  return pd.Series(series)


def evaluate_trials_with_test_and_normalization(X, y, X_test, y_test):
  y = normalize_data(y)  # 正規化
  y_test = normalize_data(y_test)  # テストデータも正規化
  rmses_train = []
  rmses_test = []
  for _ in range(N_TRIALS):
    model = train_lightgbm_full(X, y)
    rmse_train = evaluate_model(model, X, y)
    rmse_test = evaluate_on_test(model, X_test, y_test)
    rmses_train.append(rmse_train)
    rmses_test.append(rmse_test)
  return rmses_train, rmses_test


X = train_data.drop(['oss', 'Sleepiness'], axis=1)
X_test = test_data.drop(['oss', 'Sleepiness'], axis=1)

oss_rmses_train_norm, oss_rmses_test_norm = evaluate_trials_with_test_and_normalization(X, train_data['oss'], X_test, test_data['oss'])
# printResults(oss_rmses_train_norm, 'lgbm-oss-train-normalized')
printResults(oss_rmses_test_norm, 'lgbm-oss-test-normalized')
print('-------------------------')
sleepiness_rmses_train_norm, sleepiness_rmses_test_norm = evaluate_trials_with_test_and_normalization(X, train_data['Sleepiness'], X_test, test_data['Sleepiness'])
# printResults(sleepiness_rmses_train_norm, 'lgbm-sleepiness-train-normalized')
printResults(sleepiness_rmses_test_norm, 'lgbm-sleepiness-test-normalized')

[lgbm-oss-test-normalized]RMSEの平均値: 0.164027723448936
[lgbm-oss-test-normalized]RMSEの中央値: 0.164027723448936
[lgbm-oss-test-normalized]RMSEの分散: 0.0
[lgbm-oss-test-normalized]RMSEの標準偏差: 0.0
[lgbm-oss-test-normalized]RMSEの最小値: 0.164027723448936
[lgbm-oss-test-normalized]RMSEの最大値: 0.164027723448936
-------------------------
[lgbm-sleepiness-test-normalized]RMSEの平均値: 0.17198411346945977
[lgbm-sleepiness-test-normalized]RMSEの中央値: 0.17198411346945977
[lgbm-sleepiness-test-normalized]RMSEの分散: 0.0
[lgbm-sleepiness-test-normalized]RMSEの標準偏差: 0.0
[lgbm-sleepiness-test-normalized]RMSEの最小値: 0.17198411346945977
[lgbm-sleepiness-test-normalized]RMSEの最大値: 0.17198411346945977


#### 正規化 & 5 分割交差検証 & データシャッフル


In [33]:
def evaluate_with_kfold_and_normalization(X, y, n_splits=5):
  """5分割交差検証を使用してモデルを評価する関数（目的変数の正規化付き）"""
  y = normalize_data(y)
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
  rmses = []

  for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = train_lightgbm_full(X_train, y_train)
    rmse = evaluate_on_test(model, X_test, y_test)
    rmses.append(rmse)

  return rmses


# 正規化区別
outStr = ''


def shuffle_dataframe(df):
  return df.sample(frac=1.0).reset_index(drop=True)

In [34]:
print("train,testを分けて正規化\n")
train = shuffle_dataframe(get_data_from_directory(os.path.join(DATA_DIR, 'train')))
test = shuffle_dataframe(get_data_from_directory(os.path.join(DATA_DIR, 'test')))

X_train = train.drop(['oss', 'Sleepiness'], axis=1)
y_train_oss = train['oss']
y_train_sleepiness = train['Sleepiness']
X_test = test.drop(['oss', 'Sleepiness'], axis=1)
y_test_oss = test['oss']
y_test_sleepiness = test['Sleepiness']

oss_kfold_rmses_norm_divided = evaluate_with_kfold_and_normalization(X_train, y_train_oss)
printResults(oss_kfold_rmses_norm_divided, "lgbm-oss-kfold-normalized-divided")
print('-------------------------')
sleepiness_kfold_rmses_norm_divided = evaluate_with_kfold_and_normalization(X_train, y_train_sleepiness)
printResults(sleepiness_kfold_rmses_norm_divided, "lgbm-sleepiness-kfold-normalized-divided")

train,testを分けて正規化



[lgbm-oss-kfold-normalized-divided]RMSEの平均値: 0.09170989471712232
[lgbm-oss-kfold-normalized-divided]RMSEの中央値: 0.09122278495578408
[lgbm-oss-kfold-normalized-divided]RMSEの分散: 1.4917386791231595e-05
[lgbm-oss-kfold-normalized-divided]RMSEの標準偏差: 0.0038623033012998337
[lgbm-oss-kfold-normalized-divided]RMSEの最小値: 0.0860854608122519
[lgbm-oss-kfold-normalized-divided]RMSEの最大値: 0.09746616445355219
-------------------------
[lgbm-sleepiness-kfold-normalized-divided]RMSEの平均値: 0.07022711570528842
[lgbm-sleepiness-kfold-normalized-divided]RMSEの中央値: 0.07096462475628386
[lgbm-sleepiness-kfold-normalized-divided]RMSEの分散: 1.613585861798384e-05
[lgbm-sleepiness-kfold-normalized-divided]RMSEの標準偏差: 0.004016946429563611
[lgbm-sleepiness-kfold-normalized-divided]RMSEの最小値: 0.06387001375698059
[lgbm-sleepiness-kfold-normalized-divided]RMSEの最大値: 0.07619441159496756


In [35]:
print("train,testを分けずに正規化\n")
X_all = shuffle_dataframe(all_data.drop(['oss', 'Sleepiness'], axis=1))
y_all_oss = all_data['oss']
y_all_sleepiness = all_data['Sleepiness']

oss_kfold_rmses_norm = evaluate_with_kfold_and_normalization(X_all, y_all_oss)
sleepiness_kfold_rmses_norm = evaluate_with_kfold_and_normalization(X_all, y_all_sleepiness)

printResults(sleepiness_kfold_rmses_norm, 'lgbm-sleepiness-kfold-normalized-full')
print('-------------------------')
printResults(oss_kfold_rmses_norm, 'lgbm-oss-kfold-normalized-full')

train,testを分けずに正規化



[lgbm-sleepiness-kfold-normalized-full]RMSEの平均値: 0.2664253692046786
[lgbm-sleepiness-kfold-normalized-full]RMSEの中央値: 0.2658170204346171
[lgbm-sleepiness-kfold-normalized-full]RMSEの分散: 4.180828521472955e-05
[lgbm-sleepiness-kfold-normalized-full]RMSEの標準偏差: 0.006465932663949537
[lgbm-sleepiness-kfold-normalized-full]RMSEの最小値: 0.2583920746550655
[lgbm-sleepiness-kfold-normalized-full]RMSEの最大値: 0.27704439918681795
-------------------------
[lgbm-oss-kfold-normalized-full]RMSEの平均値: 0.2062093910534021
[lgbm-oss-kfold-normalized-full]RMSEの中央値: 0.20689608251461333
[lgbm-oss-kfold-normalized-full]RMSEの分散: 2.36919458549584e-05
[lgbm-oss-kfold-normalized-full]RMSEの標準偏差: 0.0048674372985132944
[lgbm-oss-kfold-normalized-full]RMSEの最小値: 0.1996066205579868
[lgbm-oss-kfold-normalized-full]RMSEの最大値: 0.2138410924594767


## 以下ゴミ

In [37]:
# 元
oss_rmses_train, oss_rmses_test = evaluate_trials_with_test(X, train_data['oss'], X_test, test_data['oss'])
printResults(oss_rmses_train, 'lgbm-oss-train')
printResults(oss_rmses_test, 'lgbm-oss-test')
###
sleepiness_rmses_train, sleepiness_rmses_test = evaluate_trials_with_test(X, train_data['Sleepiness'], X_test, test_data['Sleepiness'])
printResults(sleepiness_rmses_train, 'lgbm-sleepiness-train')
printResults(sleepiness_rmses_test, 'lgbm-sleepiness-test')

[lgbm-oss-train]RMSEの平均値: 0.12457814822031583
[lgbm-oss-train]RMSEの中央値: 0.12457814822031583
[lgbm-oss-train]RMSEの分散: 0.0
[lgbm-oss-train]RMSEの標準偏差: 0.0
[lgbm-oss-train]RMSEの最小値: 0.12457814822031583
[lgbm-oss-train]RMSEの最大値: 0.12457814822031583
[lgbm-oss-test]RMSEの平均値: 1.0616662163114088
[lgbm-oss-test]RMSEの中央値: 1.0616662163114088
[lgbm-oss-test]RMSEの分散: 0.0
[lgbm-oss-test]RMSEの標準偏差: 0.0
[lgbm-oss-test]RMSEの最小値: 1.0616662163114088
[lgbm-oss-test]RMSEの最大値: 1.0616662163114088
[lgbm-sleepiness-train]RMSEの平均値: 0.22518856302682227
[lgbm-sleepiness-train]RMSEの中央値: 0.22518856302682227
[lgbm-sleepiness-train]RMSEの分散: 0.0
[lgbm-sleepiness-train]RMSEの標準偏差: 0.0
[lgbm-sleepiness-train]RMSEの最小値: 0.22518856302682227
[lgbm-sleepiness-train]RMSEの最大値: 0.22518856302682227
[lgbm-sleepiness-test]RMSEの平均値: 2.9919831471208886
[lgbm-sleepiness-test]RMSEの中央値: 2.9919831471208886
[lgbm-sleepiness-test]RMSEの分散: 0.0
[lgbm-sleepiness-test]RMSEの標準偏差: 0.0
[lgbm-sleepiness-test]RMSEの最小値: 2.9919831471208886
[lgbm-sleep

In [38]:
# 正規化を分けてみる
all_data = pd.concat([train, test], ignore_index=True)


def normalize_data(series):
  scaler = MinMaxScaler()
  series = series.values.reshape(-1, 1)
  series = scaler.fit_transform(series)
  series = series.ravel()
  return pd.Series(series)


def evaluate_with_kfold_and_normalization(X, y, n_splits=5):
  """5分割交差検証を使用してモデルを評価する関数（目的変数の正規化付き）"""
  y = normalize_data(y)
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
  rmses = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = train_lightgbm_full(X_train, y_train)
    rmse = evaluate_on_test(model, X_test, y_test)
    rmses.append(rmse)
  return rmses


X_all = all_data.drop(['oss', 'Sleepiness'], axis=1)
y_all_oss = all_data['oss']
y_all_sleepiness = all_data['Sleepiness']

oss_kfold_rmses_norm = evaluate_with_kfold_and_normalization(X_all, y_all_oss)

sleepiness_kfold_rmses_norm = evaluate_with_kfold_and_normalization(X_all, y_all_sleepiness)
printResults(sleepiness_kfold_rmses_norm, 'lgbm-sleepiness-kfold-normalized-full')

[lgbm-sleepiness-kfold-normalized-full]RMSEの平均値: 0.07667352752696785
[lgbm-sleepiness-kfold-normalized-full]RMSEの中央値: 0.07658432781889997
[lgbm-sleepiness-kfold-normalized-full]RMSEの分散: 1.1411833473885904e-05
[lgbm-sleepiness-kfold-normalized-full]RMSEの標準偏差: 0.0033781405349520175
[lgbm-sleepiness-kfold-normalized-full]RMSEの最小値: 0.07151783616302416
[lgbm-sleepiness-kfold-normalized-full]RMSEの最大値: 0.08215868119100687


In [39]:
# 交差検証と正規化（逆変換付き）を組み合わせた関数
def evaluate_with_kfold_and_normalization_inverse(X, y, n_splits=5):
  """5分割交差検証を使用してモデルを評価する関数（目的変数の正規化と逆変換付き）"""
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
  rmses = []
  scaler = MinMaxScaler()

  for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # 正規化（訓練データでfitし、訓練データとテストデータ両方をtransform）
    y_train = y_train.values.reshape(-1, 1)
    scaler.fit(y_train)
    y_train = scaler.transform(y_train).ravel()
    y_test = scaler.transform(y_test.values.reshape(-1, 1)).ravel()

    model = train_lightgbm_full(X_train, pd.Series(y_train))  # 既存のtrain_lightgbm_full関数を使用

    # 予測値を正規化の逆変換
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions.reshape(-1, 1)).ravel()

    # RMSE計算（逆変換後の予測値と元の尺度のテストデータで）
    rmse = np.sqrt(mean_squared_error(scaler.inverse_transform(y_test.reshape(-1, 1)), predictions))
    rmses.append(rmse)

  return rmses


# 特徴量と目的変数に分割
X_all = all_data.drop(['oss', 'Sleepiness'], axis=1)
y_all_oss = all_data['oss']
y_all_sleepiness = all_data['Sleepiness']

# ossモデルの交差検証（正規化と逆変換付き）
oss_kfold_rmses_norm_inv = evaluate_with_kfold_and_normalization_inverse(X_all, y_all_oss)
printResults(oss_kfold_rmses_norm_inv, 'lgbm-oss-kfold-normalized-inverse')

# Sleepinessモデルの交差検証（正規化と逆変換付き）
sleepiness_kfold_rmses_norm_inv = evaluate_with_kfold_and_normalization_inverse(X_all, y_all_sleepiness)
printResults(sleepiness_kfold_rmses_norm_inv, 'lgbm-sleepiness-kfold-normalized-inverse')

[lgbm-oss-kfold-normalized-inverse]RMSEの平均値: 0.2994418918459029
[lgbm-oss-kfold-normalized-inverse]RMSEの中央値: 0.3010786712713193
[lgbm-oss-kfold-normalized-inverse]RMSEの分散: 0.0004235463269520362
[lgbm-oss-kfold-normalized-inverse]RMSEの標準偏差: 0.020580241178179526
[lgbm-oss-kfold-normalized-inverse]RMSEの最小値: 0.2684890026464223
[lgbm-oss-kfold-normalized-inverse]RMSEの最大値: 0.32542339853693525
[lgbm-sleepiness-kfold-normalized-inverse]RMSEの平均値: 0.6133882202157428
[lgbm-sleepiness-kfold-normalized-inverse]RMSEの中央値: 0.6126746225511998
[lgbm-sleepiness-kfold-normalized-inverse]RMSEの分散: 0.0007303573423286979
[lgbm-sleepiness-kfold-normalized-inverse]RMSEの標準偏差: 0.02702512427961614
[lgbm-sleepiness-kfold-normalized-inverse]RMSEの最小値: 0.5721426893041933
[lgbm-sleepiness-kfold-normalized-inverse]RMSEの最大値: 0.657269449528055


### GNN


In [40]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

In [41]:
def transform_to_graph_data(X, y):
  x = torch.tensor(X.values, dtype=torch.float)
  edge_index = torch.tensor([list(range(X.shape[0]-1)), list(range(1, X.shape[0]))], dtype=torch.long)
  y = torch.tensor(y.values, dtype=torch.float).view(-1, 1)
  data = Data(x=x, edge_index=edge_index, y=y)
  return data


class SimpleGNN(torch.nn.Module):
  def __init__(self, num_features):
    super(SimpleGNN, self).__init__()
    self.conv1 = GCNConv(num_features, 64)
    self.conv2 = GCNConv(64, 32)
    self.fc = torch.nn.Linear(32, 1)

  def forward(self, data):
    x, edge_index = data.x, data.edge_index
    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = self.conv2(x, edge_index)
    x = F.relu(x)
    x = self.fc(x)
    return x


def train_gnn(train_data, num_features):
  model = SimpleGNN(num_features=num_features)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    out = model(train_data)
    loss = torch.nn.MSELoss()(out, train_data.y)
    loss.backward()
    optimizer.step()
  return model


def evaluate_gnn_trials(X_train, y_train, X_test, y_test):
  train_data = transform_to_graph_data(X_train, y_train)
  test_data = transform_to_graph_data(X_test, y_test)
  gnn_rmses = []
  for _ in range(N_TRIALS):
    model = train_gnn(train_data, num_features=X_train.shape[1])
    model.eval()
    preds = model(test_data)
    rmse = np.sqrt(mean_squared_error(y_test, preds.detach().numpy()))
    gnn_rmses.append(rmse)
  return gnn_rmses

In [42]:
train = get_data_from_directory(os.path.join(DATA_DIR, 'train'))
test = get_data_from_directory(os.path.join(DATA_DIR, 'test'))

X_train = train.drop(['oss', 'Sleepiness'], axis=1)
X_test = test.drop(['oss', 'Sleepiness'], axis=1)

# ossモデル
oss_rmses = evaluate_gnn_trials(X_train, train['oss'], X_test, test['oss'])
# print(f"GNN ossモデルの平均RMSE: {np.mean(oss_rmses)}")
# print(f"GNN ossモデルのRMSEの標準偏差: {np.std(oss_rmses)}")
printResults(oss_rmses, 'gnn-oss')

# Sleepinessモデル
sleepiness_rmses = evaluate_gnn_trials(X_train, train['Sleepiness'], X_test, test['Sleepiness'])
# def compute_statistics(arr):
#     median = np.median(arr)
#     mean = np.mean(arr)
#     variance = np.var(arr)
#     std_dev = np.std(arr)
#     range_val = (np.min(arr), np.max(arr))

#     return {
#         'Median': median,
#         'Mean': mean,
#         'Variance': variance,
#         'Standard Deviation': std_dev,
#         'Range': range_val
#     }
# print(compute_statistics(sleepiness_rmses))
# print(f"GNN sleepinessモデルの平均RMSE: {np.mean(sleepiness_rmses)}")
# print(f"GNN sleepinessモデルのRMSEの標準偏差: {np.std(sleepiness_rmses)}")
printResults(sleepiness_rmses, 'gnn-sleepiness')

[gnn-oss]RMSEの平均値: 3.686751541811218
[gnn-oss]RMSEの中央値: 2.756013305613403
[gnn-oss]RMSEの分散: 5.041280566290103
[gnn-oss]RMSEの標準偏差: 2.245279618731285
[gnn-oss]RMSEの最小値: 1.7360994533111616
[gnn-oss]RMSEの最大値: 7.761015646175439
[gnn-sleepiness]RMSEの平均値: 4.612083978369974
[gnn-sleepiness]RMSEの中央値: 3.364587562382194
[gnn-sleepiness]RMSEの分散: 4.117610948268527
[gnn-sleepiness]RMSEの標準偏差: 2.029189727026166
[gnn-sleepiness]RMSEの最小値: 2.7875753257988305
[gnn-sleepiness]RMSEの最大値: 8.095109109467812


| 中央値 | 平均   | 分散   | 標準偏差 | 範囲              |
| ------ | ------ | ------ | -------- | ----------------- |
| 4.5862 | 5.0127 | 4.3706 | 2.0906   | (2.0140, 10.5504) |
